In [ ]:
import plotly.express as px
import numpy as np
import pandas as pd

In [ ]:
covid = pd.read_csv('../../data/csv/covid_us.csv')

## Group covid new cases by quarter

In [ ]:
covid['Date_reported'] = pd.to_datetime(covid['Date_reported'])
covid['quarter'] = covid['Date_reported'].dt.to_period('Q')
covid_q = covid.groupby('quarter')['New_cases'].sum().reset_index()
covid_q

,quarter,New_cases
0,2020Q1,173143
1,2020Q2,2433470
2,2020Q3,4610471
3,2020Q4,12360501
4,2021Q1,10529087
5,2021Q2,3199599
6,2021Q3,9672951
7,2021Q4,10555064
8,2022Q1,25847661
9,2022Q2,6928001


In [ ]:
# Setup - Run only once per Kernel App
%conda install openjdk -y

# install PySpark
%pip install pyspark==3.3.0

# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

Retrieving notices: ...working... done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.10.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.10.0



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - openjdk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2023.08.22 |       h06a4308_0         123 KB
    certifi-2023.7.22          |  py310h06a4308_0         153 KB
    openjdk-11.0.13            |       h87a67e3_0       341.0 MB
    ------------------------------------------------------------
                                           Total:       341.3 MB

The following NEW packages will be INSTALLED:

  openjdk            pkgs/main/

In [ ]:
# Import pyspark and build Spark session
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("PySparkApp")
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
    .config(
        "fs.s3a.aws.credentials.provider",
        "com.amazonaws.auth.ContainerCredentialsProvider",
    )
    .getOrCreate()
)

print(spark.version)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1c67278c-fb02-4ce5-a656-1aef721fe668;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 573ms :: artifacts dl 34ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	---------------------------------------------

23/11/09 01:37:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


3.3.0


## Read submission and comment dummy data

In [ ]:
%%time
s3_path = f"*"
print(f"reading combined_dummy from {s3_path}")
combined = spark.read.parquet(s3_path, header=True)
print(f"shape of the combined_dummy dataframe is {combined.count():,}x{len(combined.columns)}")

reading combined_dummy from s3a://sagemaker-us-east-1-640225923506/6000_project/combined_dummy/*


shape of the combined_dummy dataframe is 1,454,993x14
CPU times: user 36.1 ms, sys: 3.66 ms, total: 39.8 ms
Wall time: 34.4 s


In [ ]:
combined.show(5)

+---------+----------+-------------------+--------------------+-----+------+-------+-----------+--------+------+-----------+--------------+----------+-----------+
|subreddit|        id|        created_utc|                body|score|gilded|is_zoom|is_coursera|is_udemy|is_edx|is_linkedin|is_masterclass|is_youtube|is_inperson|
+---------+----------+-------------------+--------------------+-----+------+-------+-----------+--------+------+-----------+--------------+----------+-----------+
|    memes|t1_gtgpcb8|2021-04-05 15:49:18|I appreciate it. ...|   46|     0|      0|          0|       0|     0|          0|             0|         0|          0|
|    memes|t1_gtgo8ok|2021-04-05 15:49:45|Ikr! His videos a...|    1|     0|      0|          0|       0|     0|          0|             0|         0|          0|
|    memes| t3_mknezj|2021-04-05 15:49:46|You on internet e...|    3|     0|      0|          0|       0|     0|          0|             0|         0|          0|
|    memes|t1_gtgq5u8|

In [ ]:
from pyspark.sql.functions import date_format, quarter

combined = combined.withColumn("created_utc", combined["created_utc"].cast("date"))
# Extract the quarter from the "created_utc" column
combined = combined.withColumn("quarter", quarter("created_utc"))
combined = combined.withColumn("quarter", date_format("created_utc", "yyyy'Q'Q"))
# Group by "subreddit" and then by "quarter"
result = combined.groupBy("quarter","subreddit").count().orderBy("quarter")
# Show the resulting DataFrame
result.show()


+-------+------------------+------+
|quarter|         subreddit| count|
+-------+------------------+------+
| 2021Q1|          coursera|   204|
| 2021Q1|funnyonlineclasses|     9|
| 2021Q1|             memes|165847|
| 2021Q1|         teenagers| 94952|
| 2021Q1|              Zoom|  4518|
| 2021Q1|           college|  8622|
| 2021Q1|            school|  1111|
| 2021Q2|              Zoom|  3383|
| 2021Q2|             memes|127543|
| 2021Q2|          coursera|   363|
| 2021Q2|funnyonlineclasses|     4|
| 2021Q2|         teenagers| 89831|
| 2021Q2|           college|  6550|
| 2021Q2|            school|   659|
| 2021Q3|            school|   573|
| 2021Q3|              Zoom|  2978|
| 2021Q3|          coursera|   296|
| 2021Q3|         teenagers| 91571|
| 2021Q3|             memes| 85267|
| 2021Q3|funnyonlineclasses|     1|
+-------+------------------+------+
only showing top 20 rows



In [ ]:
# Save the resulting dataframe
result_df = result.toPandas()

In [ ]:
# Change the type of the quarter column
result_df['quarter'] = pd.to_datetime(result_df['quarter']).dt.to_period('Q-DEC')
# Merge it to the covid dataframe
covid_reddit = result_df.merge(covid_q,on='quarter')
covid_reddit['quarter'] = covid_reddit['quarter'].astype(str)
covid_reddit

,quarter,subreddit,count,New_cases
0,2021Q1,Zoom,4518,10529087
1,2021Q1,college,8622,10529087
2,2021Q1,funnyonlineclasses,9,10529087
3,2021Q1,teenagers,94952,10529087
4,2021Q1,school,1111,10529087
...,...,...,...,...
57,2023Q1,Zoom,1668,3285870
58,2023Q1,memes,36373,3285870
59,2023Q1,school,452,3285870
60,2023Q1,coursera,370,3285870


## Save the dataframe to csv

In [ ]:
covid_reddit.to_csv("../../data/csv/covid_reddit.csv",index=False)

## Create the scatter plot and the trend of the covid

In [1]:
import pandas as pd
covid_reddit = pd.read_csv("../../data/csv/covid_reddit.csv")

In [3]:
import plotly.subplots as sp
import plotly.graph_objects as go

# Create sub dataframes for each subreddit
zoom_data = covid_reddit[covid_reddit['subreddit'] == 'Zoom']
college_data = covid_reddit[covid_reddit['subreddit'] == 'college']
funnyonlineclasses_data = covid_reddit[covid_reddit['subreddit'] == 'funnyonlineclasses']
teenagers_data = covid_reddit[covid_reddit['subreddit'] == 'teenagers']
school_data = covid_reddit[covid_reddit['subreddit'] == 'school']
coursera_data = covid_reddit[covid_reddit['subreddit'] == 'coursera']
memes_data = covid_reddit[covid_reddit['subreddit'] == 'memes']

# Create a subplot
fig = sp.make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03,
                       row_heights=[0.2, 0.8])

# Scatter plot for each subreddit posts
scatter_trace_1 = go.Scatter(x=zoom_data['quarter'], y=zoom_data['count'],
                           mode='markers', marker=dict(size=9,color="blue"),
                           name='Zoom')
scatter_trace_2 = go.Scatter(x=college_data['quarter'], y=college_data['count'],
                           mode='markers', marker=dict(size=9,color="#ff3300"),
                           name='college')
scatter_trace_3 = go.Scatter(x=funnyonlineclasses_data['quarter'], y=funnyonlineclasses_data['count'],
                           mode='markers', marker=dict(size=9,color="green"),
                           name='funnyonlineclasses')
scatter_trace_4 = go.Scatter(x=teenagers_data['quarter'], y=teenagers_data['count'],
                           mode='markers', marker=dict(size=9,color="#9966ff"),
                           name='teenagers')
scatter_trace_5 = go.Scatter(x=school_data['quarter'], y=school_data['count'],
                           mode='markers', marker=dict(size=9,color="orange"),
                           name='school')
scatter_trace_6 = go.Scatter(x=coursera_data['quarter'], y=coursera_data['count'],
                           mode='markers', marker=dict(size=9,color="#33ccff"),
                           name='coursera')
scatter_trace_7 = go.Scatter(x=memes_data['quarter'], y=memes_data['count'],
                           mode='markers', marker=dict(size=9,color="#ff3399"),
                           name='memes')

# Line plot for Covid New Cases
line_trace = go.Scatter(x=covid_reddit['quarter'], y=covid_reddit['New_cases'],
                        mode='lines', line=dict(color='#0066cc'), name='Covid New Cases')

# Add traces to the subplots
fig.add_trace(line_trace, row=1, col=1)
fig.add_trace(scatter_trace_1, row=2, col=1)
fig.add_trace(scatter_trace_2, row=2, col=1)
fig.add_trace(scatter_trace_3, row=2, col=1)
fig.add_trace(scatter_trace_4, row=2, col=1)
fig.add_trace(scatter_trace_5, row=2, col=1)
fig.add_trace(scatter_trace_6, row=2, col=1)
fig.add_trace(scatter_trace_7, row=2, col=1)

# Update the title, legend, 
fig.update_layout(
    title="Trends of Covid new cases and Reddit posts",
    legend_title="Covid and Subreddits",
    showlegend=True,
    height=700,
    width=800
)

fig.update_yaxes(title_text="Log of New Cases",type='log', row=1, col=1)

# Update x and y-axis title for the second subplot
fig.update_yaxes(title_text="Log of Reddit Posts", type='log',row=2, col=1)
fig.update_xaxes(title_text="Time",row=2, col=1)

fig.show()
fig.write_html("../../docs/pages/covid_bubble_plot.html")